In [ ]:
from flask import Flask, jsonify, request
import requests
from bs4 import BeautifulSoup
import pandas as pd

url_listesi = [
    "https://www.hepsiburada.com/beyaz-esya-ankastreler-c-235604?sayfa=",
    "https://www.hepsiburada.com/tablet-c-3008012?sayfa=",
    "https://www.hepsiburada.com/kulakliklar-c-520?sayfa=",
    "https://www.hepsiburada.com/yazicilar-c-3013118?sayfa=",
    "https://www.hepsiburada.com/laptop-notebook-dizustu-bilgisayarlar-c-98?sayfa=",   
    "https://www.hepsiburada.com/cep-telefonlari-c-371965?sayfa=",
    "https://www.hepsiburada.com/utuler-c-155122?sayfa=",
    "https://www.hepsiburada.com/supurgeler-c-155123?sayfa=",
    "https://www.hepsiburada.com/tiras-makineleri-c-26012156?sayfa=",
    "https://www.hepsiburada.com/elektrikli-mutfak-aletleri-c-22001?sayfa=",
    "https://www.hepsiburada.com/kahve-makineleri-c-22348?sayfa=",
    "https://www.hepsiburada.com/foto-kameralari-c-2147483606?sayfa=",
    "https://www.hepsiburada.com/erkek-giyim-urunleri-c-60004641?sayfa=",
    "https://www.hepsiburada.com/bayan-giyim-urunleri-c-60004640?sayfa=",
    "https://www.hepsiburada.com/erkek-cocuk-c-205454?sayfa=",
    "https://www.hepsiburada.com/kiz-cocuk-c-205474?sayfa=",
    "https://www.hepsiburada.com/ayakkabi-c-60000117?sayfa=",
    "https://www.hepsiburada.com/bayan-ayakkabilar-c-60000116?sayfa=",
    "https://www.hepsiburada.com/erkek-botlar-c-60000125&?sayfa=",
    "https://www.hepsiburada.com/bayan-botlar-c-60000119&?sayfa=",
    "https://www.hepsiburada.com/kadin-takilari-c-80586080?sayfa=",
    "https://www.hepsiburada.com/erkek-kol-saatleri-c-25008404&?sayfa=",
    "https://www.hepsiburada.com/mobilyalar-c-18021299&?sayfa=",
    "https://www.hepsiburada.com/elektrikli-mutfak-aletleri-c-22001?sayfa=",
    "https://www.hepsiburada.com/ev-tekstili-c-2147483618&?sayfa=",
    "https://www.hepsiburada.com/ofis-mobilyalari-c-15312?sayfa="
    
    # Diğer URL'leri buraya ekleyin
]
kategori_listesi = ["Beyaz Eşya",
                    "Tablet",
                    "Kulaklık",
                    "Yazıcı",
                    "Diz Üstü Bilgisayar",
                    "Telefon","Ütü","Süpürge",
                    "Tıraş Makinesi","Elektrikli Mutfak Aletleri","Kahve Makinesi","Foto-Kamera","Erkek Giyim"
                    ,"Kadın Giyim","Erkek Bebek Giyim","Kadın Bebek Giyim","Erkek Spor Ayakkabı","Kadın Spor Ayakkabı","Erkek Bot Ayakkabı","Kadın Bot Ayakkabı","Kadın Aksesuar","Erkek Aksesuar","Mobilyalar"
                    ,"Mutfak El Aletleri","Ev Tekstil Ürünleri","Ofis Mobilyaları"]  # Kategori isimlerini buraya ekleyin
app = Flask(__name__)
gelenKategoriAdi=""
#@app.route('/api/products', methods=['GET'])
#def get_products():
 #   df = pd.read_json('mutfakevaletleri.json')
  #  products = df.to_dict(orient='records')
   # return jsonify(products)
@app.route('/api/send-string', methods=['POST'])
def receive_string():
    data = request.json
    gelenKategoriAdi = data.get('category')
    # Alınan string verisiyle yapılacak işlemleri burada gerçekleştirin
    print("Alınan kategori verisi:", gelenKategoriAdi)
    return 'OK'

@app.route('/api/<kategori>/products', methods=['GET'])
def get_products(kategori):
    filename = f"{kategori}.json"
    try:
        selected_url = None
        for url, kategori_ad in zip(url_listesi, kategori_listesi):
            if kategori_ad == kategori:
                selected_url = url
                break
        
        if selected_url is None:
            return f"Kategori bulunamadı: {kategori}", 404
        
        liste = []
        for page_number in range(1, 2):
            HEADERS = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36 Edg/108.0.1462.54'}
            r = requests.get(f"{selected_url}{page_number}", headers=HEADERS)
            soup = BeautifulSoup(r.content, "lxml")
            st1 = soup.find("div", attrs={"class": "productListContent-pXUkO4iHa51o_17CBibU"})
            st2 = st1.find("ul", attrs={"class": "productListContent-frGrtf5XrVXRwJ05HUfU productListContent-rEYj2_8SETJUeqNhyzSm"})
            st3 = st2.find_all("li", attrs={"class": "productListContent-zAP0Y5msy8OHn5z7T_K_"})
            
            for detaylar in st3:
                link_sonu = detaylar.a.get("href")
                link_basi ="https://www.hepsiburada.com"
                if link_sonu.startswith("https"):
                    continue
                else:
                    link = link_basi + link_sonu
                                
                r1 = requests.get(link, headers=HEADERS)
                soup1 = BeautifulSoup(r1.content, "lxml")
                urun_ad = soup1.find("h1", attrs={"id": "product-name"}).text.strip()
                urun_markaSt = soup1.find("span", attrs={"class": "brand-name"})
                urun_marka = urun_markaSt.find("a").text
                urunfiyat = soup1.find("span", attrs={"data-bind": "markupText:'currentPriceBeforePoint'"}).text
                urunfiyat = int(urunfiyat.replace(".", ""))
                
                urunImageSoup = soup1.find("picture", attrs={"index": "0"})
                urunUrl123 = urunImageSoup.find("img")
                urunImageUrl = urunUrl123.get("src")
                
                liste.append([link, urun_ad, urun_marka, urunfiyat, urunImageUrl])
        
        df = pd.DataFrame(liste)
        df.columns = ["UrunLink", "UrunAd", "UrunMarka", "UrunFiyat", "UrunResim"]
        df.to_json(f"{kategori}.json", orient="records")
        
        products = df.to_dict(orient='records')
        return jsonify(products)
    
    except FileNotFoundError:
        return f"JSON dosyası bulunamadı: {filename}", 404


if __name__ == '__main__':
    app.run(port=8000)


